##### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier

##### READ DATA AND VISUALIZATION

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.dropna(subset=["Embarked"], axis=0, inplace=True)
test.Fare = test.Fare.fillna(7.8)
train = train.loc[train.Fare<300, :]

titanic = pd.concat([train, test], axis=0, ignore_index=True)

##### FEATURE ENG

In [3]:
titanic["Fare_Log"] = np.log1p(titanic.Fare) 
titanic["Fare_Log_Bin"] = pd.cut(titanic["Fare_Log"], bins=[-0.5,1,2.4,3,4,5,7], labels=["f1","f2","f3","f4","f5","f6"])

In [4]:
titanic["Cabin"] = titanic.Cabin.fillna("N")
#titanic["Cabin_Number"] = titanic.Cabin.map(lambda x: x.strip()[1:3] if len(x.split(" ")[0])==3 else x.strip()[1:4]).map(lambda x: )
titanic["IsCabin"] = titanic.Cabin.map(lambda x: 0 if x == "N" else 1)

In [5]:
titanic["Title"] = titanic.Name.map(lambda x: x.split(",")[1].split(".")[0].strip()).replace(["Capt", "Col", "Don", "Dr",
                                    "Jonkheer", "Major","Rev","Sir"], "Others").replace(["Dona", "Lady", "Mlle", "Mme",
                                                                                         "Ms", "the Countess"],"Mrs")
titanic["Surname"] = titanic.Name.map(lambda x: x.split(",")[0].strip())
titanic["Family"] = titanic.SibSp + titanic.Parch + 1
titanic["Family_Bin"] = titanic.Family.replace([2,3,4],2).replace([5,6,7,8,11], 3)
titanic["IsMarige"] = titanic.Parch.map(lambda x: 1 if x > 0 else 0)

In [6]:
titanic["Ticket_First"] = titanic.Ticket.map(lambda x: x[0]).replace(["4","5","6","7","8","A","L","W","9"], "4").replace("F","C")
titanic["Ticket_Len"] = titanic.Ticket.map(lambda x: len(x))

##### AGE FILL NA

In [7]:
titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin", "Fare_Log_Bin", "Embarked", "IsMarige"])["Age"].apply(lambda x: x.fillna(x.mean()))
titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin", "Fare_Log_Bin", "Embarked"])["Age"].apply(lambda x: x.fillna(x.mean()))
titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin", "Fare_Log_Bin"])["Age"].apply(lambda x: x.fillna(x.mean()))
titanic["Age"] = titanic.groupby(["Sex", "Pclass", "IsCabin", "Family_Bin"])["Age"].apply(lambda x: x.fillna(x.mean()))

titanic["Age_Bin"] = pd.cut(titanic.Age, bins=[-0.5,10,17,25,30,40,60,85], labels=["age1","age2","age3","age4","age5","age6","age7"])
titanic["Age_Log"] = np.log1p(titanic.Age)

# MODELING

In [8]:
Id = test.PassengerId
target = train.Survived
split_index = len(train)

In [9]:
DATA = titanic.drop(["PassengerId", "Survived", "Name", "SibSp", "Parch", "Ticket", "Fare", "Surname",
                     "Family", "Ticket_Len", "Age_Bin", "Age_Log", "Cabin", "Fare_Log_Bin"], axis=1)

In [10]:
DF = pd.get_dummies(data=DATA, columns=["Pclass", "Sex", "Embarked", "IsCabin", "Title", "Family_Bin", "IsMarige",
                                        "Ticket_First"], drop_first=True)

In [11]:
#icq = 1.5*(36.125-21)
#icq_interval = [0, 36.125 + icq]
#DF.Age = (DF.Age - DF.Age.median())/icq_interval[1]

In [12]:
#icq = 1.5*(3.465736-2.185579)
#icq_interval = [2.185579 - icq, 3.465736 + icq]
#DF.Fare_Log = (DF.Fare_Log - DF.Fare_Log.median())/(icq_interval[1]-icq_interval[0])

In [13]:
DF.Age = (DF.Age - DF.Age.min())/(DF.Age.max()-DF.Age.min())
DF.Fare_Log = (DF.Fare_Log - DF.Fare_Log.min())/(DF.Fare_Log.max()-DF.Fare_Log.min())

In [14]:
pca = PCA(13)
DF_pca = pca.fit_transform(DF)
print(pca.explained_variance_ratio_.sum())

0.9566759095097307


In [15]:
X_TRAIN = DF.iloc[:split_index]
X_TEST = DF.iloc[split_index:]

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_TRAIN, target, test_size=0.25, random_state=np.random.randint(1,300))

##### 0 Models

In [17]:
tree = DecisionTreeClassifier()
forest = RandomForestClassifier()
gb = GradientBoostingClassifier()
xgb = XGBClassifier()
log = LogisticRegression()
knn = KNeighborsClassifier()
nb = GaussianNB()
cb = CatBoostClassifier()
lgbm = LGBMClassifier()
svm = SVC(probability=True)

Models = [tree, forest, gb, xgb, knn, nb, cb, lgbm, svm]

In [18]:
res = {}
for i, clf in zip(["Tree", "Forest", "Gb", "Xgb", "Log", "knn", "naive", "catboost", "lgbm", "svm"],Models):
    res[i] = cross_val_score(clf, X_TRAIN, target, scoring="accuracy", cv=20).mean()

Learning rate set to 0.009568
0:	learn: 0.6858955	total: 141ms	remaining: 2m 20s
1:	learn: 0.6805150	total: 145ms	remaining: 1m 12s
2:	learn: 0.6739143	total: 148ms	remaining: 49.3s
3:	learn: 0.6680770	total: 151ms	remaining: 37.7s
4:	learn: 0.6618591	total: 154ms	remaining: 30.7s
5:	learn: 0.6557280	total: 158ms	remaining: 26.2s
6:	learn: 0.6496364	total: 161ms	remaining: 22.9s
7:	learn: 0.6434953	total: 165ms	remaining: 20.4s
8:	learn: 0.6378044	total: 168ms	remaining: 18.5s
9:	learn: 0.6315609	total: 172ms	remaining: 17s
10:	learn: 0.6257135	total: 175ms	remaining: 15.7s
11:	learn: 0.6203924	total: 178ms	remaining: 14.7s
12:	learn: 0.6155633	total: 181ms	remaining: 13.8s
13:	learn: 0.6108053	total: 185ms	remaining: 13s
14:	learn: 0.6054991	total: 187ms	remaining: 12.3s
15:	learn: 0.5998548	total: 189ms	remaining: 11.6s
16:	learn: 0.5954506	total: 192ms	remaining: 11.1s
17:	learn: 0.5910022	total: 195ms	remaining: 10.6s
18:	learn: 0.5861611	total: 196ms	remaining: 10.1s
19:	learn: 0.

227:	learn: 0.3456176	total: 534ms	remaining: 1.81s
228:	learn: 0.3454125	total: 536ms	remaining: 1.8s
229:	learn: 0.3451910	total: 537ms	remaining: 1.8s
230:	learn: 0.3448561	total: 539ms	remaining: 1.79s
231:	learn: 0.3444542	total: 540ms	remaining: 1.79s
232:	learn: 0.3441864	total: 542ms	remaining: 1.78s
233:	learn: 0.3438472	total: 543ms	remaining: 1.78s
234:	learn: 0.3435479	total: 545ms	remaining: 1.77s
235:	learn: 0.3433423	total: 547ms	remaining: 1.77s
236:	learn: 0.3429507	total: 549ms	remaining: 1.77s
237:	learn: 0.3424968	total: 551ms	remaining: 1.76s
238:	learn: 0.3423519	total: 553ms	remaining: 1.76s
239:	learn: 0.3421341	total: 554ms	remaining: 1.75s
240:	learn: 0.3419179	total: 556ms	remaining: 1.75s
241:	learn: 0.3415877	total: 557ms	remaining: 1.75s
242:	learn: 0.3414430	total: 559ms	remaining: 1.74s
243:	learn: 0.3412319	total: 561ms	remaining: 1.74s
244:	learn: 0.3409996	total: 562ms	remaining: 1.73s
245:	learn: 0.3406683	total: 564ms	remaining: 1.73s
246:	learn: 0.

467:	learn: 0.2990329	total: 885ms	remaining: 1.01s
468:	learn: 0.2988894	total: 887ms	remaining: 1s
469:	learn: 0.2988015	total: 888ms	remaining: 1s
470:	learn: 0.2987475	total: 890ms	remaining: 1000ms
471:	learn: 0.2986398	total: 891ms	remaining: 997ms
472:	learn: 0.2984298	total: 893ms	remaining: 995ms
473:	learn: 0.2982343	total: 895ms	remaining: 993ms
474:	learn: 0.2980284	total: 897ms	remaining: 991ms
475:	learn: 0.2979898	total: 898ms	remaining: 988ms
476:	learn: 0.2979741	total: 899ms	remaining: 985ms
477:	learn: 0.2976461	total: 900ms	remaining: 983ms
478:	learn: 0.2975301	total: 902ms	remaining: 981ms
479:	learn: 0.2973584	total: 903ms	remaining: 978ms
480:	learn: 0.2972309	total: 905ms	remaining: 976ms
481:	learn: 0.2970829	total: 906ms	remaining: 974ms
482:	learn: 0.2968682	total: 907ms	remaining: 971ms
483:	learn: 0.2966945	total: 909ms	remaining: 969ms
484:	learn: 0.2966441	total: 910ms	remaining: 966ms
485:	learn: 0.2965087	total: 911ms	remaining: 964ms
486:	learn: 0.296

660:	learn: 0.2718898	total: 1.24s	remaining: 636ms
661:	learn: 0.2718106	total: 1.24s	remaining: 634ms
662:	learn: 0.2715962	total: 1.26s	remaining: 639ms
663:	learn: 0.2714447	total: 1.26s	remaining: 638ms
664:	learn: 0.2712841	total: 1.26s	remaining: 636ms
665:	learn: 0.2710413	total: 1.27s	remaining: 635ms
666:	learn: 0.2709455	total: 1.27s	remaining: 634ms
667:	learn: 0.2708379	total: 1.27s	remaining: 633ms
668:	learn: 0.2707428	total: 1.28s	remaining: 632ms
669:	learn: 0.2705973	total: 1.28s	remaining: 630ms
670:	learn: 0.2704713	total: 1.28s	remaining: 629ms
671:	learn: 0.2703608	total: 1.28s	remaining: 628ms
672:	learn: 0.2702634	total: 1.29s	remaining: 626ms
673:	learn: 0.2701617	total: 1.29s	remaining: 625ms
674:	learn: 0.2700413	total: 1.29s	remaining: 624ms
675:	learn: 0.2699234	total: 1.3s	remaining: 622ms
676:	learn: 0.2697751	total: 1.3s	remaining: 621ms
677:	learn: 0.2696107	total: 1.3s	remaining: 620ms
678:	learn: 0.2694887	total: 1.31s	remaining: 619ms
679:	learn: 0.2

833:	learn: 0.2503261	total: 1.77s	remaining: 352ms
834:	learn: 0.2501833	total: 1.77s	remaining: 350ms
835:	learn: 0.2500826	total: 1.77s	remaining: 348ms
836:	learn: 0.2499166	total: 1.77s	remaining: 346ms
837:	learn: 0.2498133	total: 1.78s	remaining: 344ms
838:	learn: 0.2497030	total: 1.78s	remaining: 342ms
839:	learn: 0.2495837	total: 1.78s	remaining: 340ms
840:	learn: 0.2494296	total: 1.78s	remaining: 337ms
841:	learn: 0.2492273	total: 1.79s	remaining: 335ms
842:	learn: 0.2491568	total: 1.79s	remaining: 333ms
843:	learn: 0.2490317	total: 1.79s	remaining: 331ms
844:	learn: 0.2489663	total: 1.79s	remaining: 329ms
845:	learn: 0.2487199	total: 1.79s	remaining: 327ms
846:	learn: 0.2486526	total: 1.8s	remaining: 324ms
847:	learn: 0.2484302	total: 1.8s	remaining: 322ms
848:	learn: 0.2483441	total: 1.8s	remaining: 320ms
849:	learn: 0.2483114	total: 1.8s	remaining: 318ms
850:	learn: 0.2481670	total: 1.8s	remaining: 316ms
851:	learn: 0.2480369	total: 1.8s	remaining: 313ms
852:	learn: 0.2479

...

0:	learn: 0.6861761	total: 3.15ms	remaining: 3.15s
1:	learn: 0.6812730	total: 6.57ms	remaining: 3.28s
2:	learn: 0.6750298	total: 9.99ms	remaining: 3.32s
3:	learn: 0.6692900	total: 13ms	remaining: 3.24s
4:	learn: 0.6632512	total: 16.2ms	remaining: 3.23s
5:	learn: 0.6574400	total: 19.4ms	remaining: 3.22s
6:	learn: 0.6514175	total: 22.5ms	remaining: 3.2s
7:	learn: 0.6454933	total: 24.7ms	remaining: 3.06s
8:	learn: 0.6397560	total: 27.2ms	remaining: 3s
9:	learn: 0.6339292	total: 29.8ms	remaining: 2.95s
10:	learn: 0.6284367	total: 32.8ms	remaining: 2.95s
11:	learn: 0.6234932	total: 35.6ms	remaining: 2.93s
12:	learn: 0.6187002	total: 37.6ms	remaining: 2.85s
13:	learn: 0.6146643	total: 40.3ms	remaining: 2.84s
14:	learn: 0.6093920	total: 42.4ms	remaining: 2.78s
15:	learn: 0.6051738	total: 43.8ms	remaining: 2.7s
16:	learn: 0.6009608	total: 45.5ms	remaining: 2.63s
17:	learn: 0.5965569	total: 46.9ms	remaining: 2.56s
18:	learn: 0.5917548	total: 48.4ms	remaining: 2.5s
19:	learn: 0.5877172	total: 49

197:	learn: 0.3663355	total: 375ms	remaining: 1.52s
198:	learn: 0.3660821	total: 379ms	remaining: 1.52s
199:	learn: 0.3656281	total: 382ms	remaining: 1.53s
200:	learn: 0.3651908	total: 385ms	remaining: 1.53s
201:	learn: 0.3649194	total: 389ms	remaining: 1.53s
202:	learn: 0.3646319	total: 392ms	remaining: 1.54s
203:	learn: 0.3643263	total: 394ms	remaining: 1.54s
204:	learn: 0.3638699	total: 396ms	remaining: 1.54s
205:	learn: 0.3634752	total: 399ms	remaining: 1.53s
206:	learn: 0.3631100	total: 401ms	remaining: 1.54s
207:	learn: 0.3627681	total: 404ms	remaining: 1.54s
208:	learn: 0.3624542	total: 406ms	remaining: 1.53s
209:	learn: 0.3620648	total: 409ms	remaining: 1.54s
210:	learn: 0.3620093	total: 410ms	remaining: 1.53s
211:	learn: 0.3615474	total: 412ms	remaining: 1.53s
212:	learn: 0.3612694	total: 415ms	remaining: 1.53s
213:	learn: 0.3609442	total: 417ms	remaining: 1.53s
214:	learn: 0.3606628	total: 419ms	remaining: 1.53s
215:	learn: 0.3603876	total: 421ms	remaining: 1.53s
216:	learn: 

400:	learn: 0.3211151	total: 714ms	remaining: 1.06s
401:	learn: 0.3209798	total: 715ms	remaining: 1.06s
402:	learn: 0.3209071	total: 717ms	remaining: 1.06s
403:	learn: 0.3207531	total: 718ms	remaining: 1.06s
404:	learn: 0.3206394	total: 720ms	remaining: 1.06s
405:	learn: 0.3205260	total: 722ms	remaining: 1.05s
406:	learn: 0.3203432	total: 723ms	remaining: 1.05s
407:	learn: 0.3201356	total: 725ms	remaining: 1.05s
408:	learn: 0.3199359	total: 726ms	remaining: 1.05s
409:	learn: 0.3197622	total: 728ms	remaining: 1.05s
410:	learn: 0.3195709	total: 729ms	remaining: 1.04s
411:	learn: 0.3194289	total: 731ms	remaining: 1.04s
412:	learn: 0.3192665	total: 733ms	remaining: 1.04s
413:	learn: 0.3191228	total: 734ms	remaining: 1.04s
414:	learn: 0.3191039	total: 735ms	remaining: 1.03s
415:	learn: 0.3189142	total: 737ms	remaining: 1.03s
416:	learn: 0.3186547	total: 739ms	remaining: 1.03s
417:	learn: 0.3185204	total: 741ms	remaining: 1.03s
418:	learn: 0.3184228	total: 742ms	remaining: 1.03s
419:	learn: 

624:	learn: 0.2894421	total: 1.05s	remaining: 633ms
625:	learn: 0.2893836	total: 1.06s	remaining: 631ms
626:	learn: 0.2892186	total: 1.06s	remaining: 629ms
627:	learn: 0.2891536	total: 1.06s	remaining: 628ms
628:	learn: 0.2890943	total: 1.06s	remaining: 626ms
629:	learn: 0.2890182	total: 1.06s	remaining: 624ms
630:	learn: 0.2886534	total: 1.06s	remaining: 623ms
631:	learn: 0.2885231	total: 1.07s	remaining: 621ms
632:	learn: 0.2883400	total: 1.07s	remaining: 619ms
633:	learn: 0.2881436	total: 1.07s	remaining: 617ms
634:	learn: 0.2880230	total: 1.07s	remaining: 615ms
635:	learn: 0.2879106	total: 1.07s	remaining: 614ms
636:	learn: 0.2878192	total: 1.07s	remaining: 612ms
637:	learn: 0.2876341	total: 1.08s	remaining: 611ms
638:	learn: 0.2875150	total: 1.08s	remaining: 609ms
639:	learn: 0.2873074	total: 1.08s	remaining: 607ms
640:	learn: 0.2872300	total: 1.08s	remaining: 605ms
641:	learn: 0.2870427	total: 1.08s	remaining: 604ms
642:	learn: 0.2868680	total: 1.08s	remaining: 602ms
643:	learn: 

826:	learn: 0.2633168	total: 1.4s	remaining: 293ms
827:	learn: 0.2630963	total: 1.4s	remaining: 291ms
828:	learn: 0.2630215	total: 1.41s	remaining: 291ms
829:	learn: 0.2629363	total: 1.41s	remaining: 289ms
830:	learn: 0.2626153	total: 1.41s	remaining: 288ms
831:	learn: 0.2625665	total: 1.42s	remaining: 286ms
832:	learn: 0.2623961	total: 1.42s	remaining: 284ms
833:	learn: 0.2622723	total: 1.42s	remaining: 283ms
834:	learn: 0.2621893	total: 1.42s	remaining: 281ms
835:	learn: 0.2620938	total: 1.42s	remaining: 279ms
836:	learn: 0.2619871	total: 1.43s	remaining: 278ms
837:	learn: 0.2618305	total: 1.43s	remaining: 276ms
838:	learn: 0.2617984	total: 1.43s	remaining: 275ms
839:	learn: 0.2616944	total: 1.43s	remaining: 273ms
840:	learn: 0.2615742	total: 1.43s	remaining: 271ms
841:	learn: 0.2614947	total: 1.44s	remaining: 269ms
842:	learn: 0.2614511	total: 1.44s	remaining: 268ms
843:	learn: 0.2613788	total: 1.44s	remaining: 266ms
844:	learn: 0.2612835	total: 1.44s	remaining: 264ms
845:	learn: 0.

In [19]:
keys = list(res.keys())
values = list(res.values())
df_scores1 = pd.DataFrame({"Models":keys,"CV_Scores":values})

In [20]:
df_scores1

,Models,CV_Scores
0,Tree,0.776717
1,Forest,0.816061
2,Gb,0.829722
3,Xgb,0.821944
4,Log,0.797045
5,knn,0.811616
6,naive,0.830859
7,catboost,0.828611
8,lgbm,0.830783


##### Little Models

In [21]:
tree = DecisionTreeClassifier(max_depth=6)
forest = RandomForestClassifier(max_depth=6)
gb = GradientBoostingClassifier(max_depth=6)
xgb = XGBClassifier(max_depth=6)
log = LogisticRegression()
knn = KNeighborsClassifier()
nb = GaussianNB()
cb = CatBoostClassifier(max_depth=6)
lgbm = LGBMClassifier(max_depth=6)
svm = SVC(probability=True)

Models = [tree, forest, gb, xgb, knn, nb, cb, lgbm, svm]

In [22]:
res = {}
for i, clf in zip(["Tree", "Forest", "Gb", "Xgb", "Log", "knn", "naive", "catboost", "lgbm", "svm"],Models):
    res[i] = cross_val_score(clf, X_TRAIN, target, scoring="accuracy", cv=20).mean()

Learning rate set to 0.009568
0:	learn: 0.6858955	total: 2.72ms	remaining: 2.72s
1:	learn: 0.6805150	total: 5.92ms	remaining: 2.95s
2:	learn: 0.6739143	total: 9.2ms	remaining: 3.06s
3:	learn: 0.6680770	total: 12.5ms	remaining: 3.11s
4:	learn: 0.6618591	total: 15.7ms	remaining: 3.12s
5:	learn: 0.6557280	total: 18.7ms	remaining: 3.1s
6:	learn: 0.6496364	total: 21.2ms	remaining: 3.01s
7:	learn: 0.6434953	total: 23.4ms	remaining: 2.9s
8:	learn: 0.6378044	total: 25.7ms	remaining: 2.83s
9:	learn: 0.6315609	total: 28.1ms	remaining: 2.79s
10:	learn: 0.6257135	total: 30.3ms	remaining: 2.73s
11:	learn: 0.6203924	total: 31.9ms	remaining: 2.63s
12:	learn: 0.6155633	total: 34ms	remaining: 2.58s
13:	learn: 0.6108053	total: 36.3ms	remaining: 2.56s
14:	learn: 0.6054991	total: 37.6ms	remaining: 2.47s
15:	learn: 0.5998548	total: 38.9ms	remaining: 2.39s
16:	learn: 0.5954506	total: 40.5ms	remaining: 2.34s
17:	learn: 0.5910022	total: 41.8ms	remaining: 2.28s
18:	learn: 0.5861611	total: 43.1ms	remaining: 2.2

202:	learn: 0.3542116	total: 340ms	remaining: 1.33s
203:	learn: 0.3536237	total: 341ms	remaining: 1.33s
204:	learn: 0.3531468	total: 343ms	remaining: 1.33s
205:	learn: 0.3531032	total: 344ms	remaining: 1.33s
206:	learn: 0.3527348	total: 346ms	remaining: 1.32s
207:	learn: 0.3524423	total: 347ms	remaining: 1.32s
208:	learn: 0.3519402	total: 349ms	remaining: 1.32s
209:	learn: 0.3516029	total: 351ms	remaining: 1.32s
210:	learn: 0.3512230	total: 354ms	remaining: 1.32s
211:	learn: 0.3508428	total: 356ms	remaining: 1.32s
212:	learn: 0.3505087	total: 358ms	remaining: 1.32s
213:	learn: 0.3501528	total: 359ms	remaining: 1.32s
214:	learn: 0.3498204	total: 361ms	remaining: 1.32s
215:	learn: 0.3495393	total: 363ms	remaining: 1.32s
216:	learn: 0.3491898	total: 364ms	remaining: 1.31s
217:	learn: 0.3486642	total: 371ms	remaining: 1.33s
218:	learn: 0.3483862	total: 373ms	remaining: 1.33s
219:	learn: 0.3480418	total: 374ms	remaining: 1.33s
220:	learn: 0.3477210	total: 376ms	remaining: 1.32s
221:	learn: 

438:	learn: 0.3032541	total: 690ms	remaining: 882ms
439:	learn: 0.3031289	total: 692ms	remaining: 880ms
440:	learn: 0.3030499	total: 693ms	remaining: 879ms
441:	learn: 0.3028270	total: 695ms	remaining: 877ms
442:	learn: 0.3027461	total: 696ms	remaining: 876ms
443:	learn: 0.3026010	total: 698ms	remaining: 874ms
444:	learn: 0.3022947	total: 700ms	remaining: 873ms
445:	learn: 0.3021688	total: 702ms	remaining: 871ms
446:	learn: 0.3020036	total: 703ms	remaining: 870ms
447:	learn: 0.3018932	total: 705ms	remaining: 869ms
448:	learn: 0.3016411	total: 706ms	remaining: 867ms
449:	learn: 0.3014689	total: 708ms	remaining: 865ms
450:	learn: 0.3012796	total: 709ms	remaining: 864ms
451:	learn: 0.3010123	total: 711ms	remaining: 862ms
452:	learn: 0.3008951	total: 712ms	remaining: 860ms
453:	learn: 0.3007744	total: 714ms	remaining: 858ms
454:	learn: 0.3006096	total: 715ms	remaining: 857ms
455:	learn: 0.3005298	total: 716ms	remaining: 855ms
456:	learn: 0.3003921	total: 718ms	remaining: 853ms
457:	learn: 

682:	learn: 0.2688550	total: 1.04s	remaining: 483ms
683:	learn: 0.2687034	total: 1.04s	remaining: 482ms
684:	learn: 0.2684905	total: 1.04s	remaining: 480ms
685:	learn: 0.2683951	total: 1.04s	remaining: 479ms
686:	learn: 0.2683106	total: 1.05s	remaining: 477ms
687:	learn: 0.2681899	total: 1.05s	remaining: 476ms
688:	learn: 0.2680281	total: 1.05s	remaining: 475ms
689:	learn: 0.2679424	total: 1.05s	remaining: 474ms
690:	learn: 0.2678777	total: 1.05s	remaining: 472ms
691:	learn: 0.2677744	total: 1.06s	remaining: 471ms
692:	learn: 0.2676186	total: 1.06s	remaining: 469ms
693:	learn: 0.2673740	total: 1.06s	remaining: 468ms
694:	learn: 0.2672911	total: 1.06s	remaining: 466ms
695:	learn: 0.2671792	total: 1.06s	remaining: 465ms
696:	learn: 0.2670921	total: 1.06s	remaining: 463ms
697:	learn: 0.2669949	total: 1.07s	remaining: 462ms
698:	learn: 0.2669182	total: 1.07s	remaining: 460ms
699:	learn: 0.2667180	total: 1.07s	remaining: 459ms
700:	learn: 0.2666684	total: 1.07s	remaining: 457ms
701:	learn: 

920:	learn: 0.2400156	total: 1.39s	remaining: 119ms
921:	learn: 0.2399575	total: 1.39s	remaining: 118ms
922:	learn: 0.2398910	total: 1.39s	remaining: 116ms
923:	learn: 0.2398403	total: 1.4s	remaining: 115ms
924:	learn: 0.2396838	total: 1.4s	remaining: 113ms
925:	learn: 0.2396064	total: 1.4s	remaining: 112ms
926:	learn: 0.2394962	total: 1.4s	remaining: 110ms
927:	learn: 0.2394560	total: 1.4s	remaining: 109ms
928:	learn: 0.2392763	total: 1.4s	remaining: 107ms
929:	learn: 0.2391872	total: 1.41s	remaining: 106ms
930:	learn: 0.2391078	total: 1.41s	remaining: 104ms
931:	learn: 0.2389760	total: 1.41s	remaining: 103ms
932:	learn: 0.2388732	total: 1.41s	remaining: 101ms
933:	learn: 0.2388028	total: 1.41s	remaining: 99.8ms
934:	learn: 0.2387726	total: 1.41s	remaining: 98.3ms
935:	learn: 0.2386253	total: 1.42s	remaining: 96.8ms
936:	learn: 0.2385324	total: 1.42s	remaining: 95.2ms
937:	learn: 0.2384946	total: 1.42s	remaining: 93.7ms
938:	learn: 0.2382486	total: 1.42s	remaining: 92.2ms
939:	learn: 

...

183:	learn: 0.3716192	total: 354ms	remaining: 1.57s
184:	learn: 0.3711343	total: 357ms	remaining: 1.57s
185:	learn: 0.3707315	total: 359ms	remaining: 1.57s
186:	learn: 0.3703220	total: 362ms	remaining: 1.57s
187:	learn: 0.3699485	total: 365ms	remaining: 1.58s
188:	learn: 0.3695142	total: 367ms	remaining: 1.57s
189:	learn: 0.3691396	total: 369ms	remaining: 1.57s
190:	learn: 0.3689841	total: 370ms	remaining: 1.57s
191:	learn: 0.3685812	total: 372ms	remaining: 1.57s
192:	learn: 0.3682526	total: 374ms	remaining: 1.56s
193:	learn: 0.3679029	total: 375ms	remaining: 1.56s
194:	learn: 0.3674340	total: 377ms	remaining: 1.55s
195:	learn: 0.3671628	total: 378ms	remaining: 1.55s
196:	learn: 0.3667063	total: 380ms	remaining: 1.55s
197:	learn: 0.3663355	total: 382ms	remaining: 1.54s
198:	learn: 0.3660821	total: 383ms	remaining: 1.54s
199:	learn: 0.3656281	total: 385ms	remaining: 1.54s
200:	learn: 0.3651908	total: 386ms	remaining: 1.53s
201:	learn: 0.3649194	total: 387ms	remaining: 1.53s
202:	learn: 

424:	learn: 0.3177272	total: 704ms	remaining: 952ms
425:	learn: 0.3176279	total: 705ms	remaining: 951ms
426:	learn: 0.3174506	total: 707ms	remaining: 949ms
427:	learn: 0.3172226	total: 709ms	remaining: 948ms
428:	learn: 0.3171093	total: 711ms	remaining: 947ms
429:	learn: 0.3168863	total: 713ms	remaining: 945ms
430:	learn: 0.3168672	total: 714ms	remaining: 942ms
431:	learn: 0.3167584	total: 716ms	remaining: 941ms
432:	learn: 0.3166700	total: 718ms	remaining: 940ms
433:	learn: 0.3164359	total: 719ms	remaining: 938ms
434:	learn: 0.3162479	total: 721ms	remaining: 936ms
435:	learn: 0.3160851	total: 722ms	remaining: 934ms
436:	learn: 0.3159129	total: 724ms	remaining: 933ms
437:	learn: 0.3158337	total: 726ms	remaining: 931ms
438:	learn: 0.3157263	total: 727ms	remaining: 929ms
439:	learn: 0.3156166	total: 729ms	remaining: 927ms
440:	learn: 0.3153845	total: 730ms	remaining: 926ms
441:	learn: 0.3152184	total: 732ms	remaining: 924ms
442:	learn: 0.3151215	total: 733ms	remaining: 921ms
443:	learn: 

600:	learn: 0.2921804	total: 1.06s	remaining: 702ms
601:	learn: 0.2918458	total: 1.06s	remaining: 701ms
602:	learn: 0.2918299	total: 1.06s	remaining: 699ms
603:	learn: 0.2916783	total: 1.06s	remaining: 698ms
604:	learn: 0.2916702	total: 1.06s	remaining: 696ms
605:	learn: 0.2915920	total: 1.07s	remaining: 695ms
606:	learn: 0.2914576	total: 1.07s	remaining: 694ms
607:	learn: 0.2912252	total: 1.07s	remaining: 692ms
608:	learn: 0.2911208	total: 1.07s	remaining: 690ms
609:	learn: 0.2910930	total: 1.08s	remaining: 688ms
610:	learn: 0.2909707	total: 1.08s	remaining: 687ms
611:	learn: 0.2908732	total: 1.08s	remaining: 685ms
612:	learn: 0.2906893	total: 1.08s	remaining: 683ms
613:	learn: 0.2906851	total: 1.08s	remaining: 680ms
614:	learn: 0.2904943	total: 1.08s	remaining: 679ms
615:	learn: 0.2904740	total: 1.08s	remaining: 676ms
616:	learn: 0.2903277	total: 1.09s	remaining: 674ms
617:	learn: 0.2901977	total: 1.09s	remaining: 672ms
618:	learn: 0.2900770	total: 1.09s	remaining: 671ms
619:	learn: 

791:	learn: 0.2680327	total: 1.57s	remaining: 414ms
792:	learn: 0.2679806	total: 1.58s	remaining: 412ms
793:	learn: 0.2679416	total: 1.58s	remaining: 410ms
794:	learn: 0.2678874	total: 1.58s	remaining: 408ms
795:	learn: 0.2677438	total: 1.59s	remaining: 407ms
796:	learn: 0.2676583	total: 1.59s	remaining: 405ms
797:	learn: 0.2675354	total: 1.59s	remaining: 403ms
798:	learn: 0.2674563	total: 1.59s	remaining: 401ms
799:	learn: 0.2672802	total: 1.59s	remaining: 399ms
800:	learn: 0.2671782	total: 1.6s	remaining: 397ms
801:	learn: 0.2671045	total: 1.6s	remaining: 395ms
802:	learn: 0.2669798	total: 1.6s	remaining: 393ms
803:	learn: 0.2668941	total: 1.61s	remaining: 392ms
804:	learn: 0.2667258	total: 1.61s	remaining: 390ms
805:	learn: 0.2665795	total: 1.61s	remaining: 387ms
806:	learn: 0.2665757	total: 1.61s	remaining: 385ms
807:	learn: 0.2664937	total: 1.61s	remaining: 383ms
808:	learn: 0.2662692	total: 1.61s	remaining: 382ms
809:	learn: 0.2660064	total: 1.62s	remaining: 380ms
810:	learn: 0.2

971:	learn: 0.2475046	total: 2.1s	remaining: 60.6ms
972:	learn: 0.2474692	total: 2.1s	remaining: 58.4ms
973:	learn: 0.2474054	total: 2.11s	remaining: 56.2ms
974:	learn: 0.2473124	total: 2.11s	remaining: 54.1ms
975:	learn: 0.2471621	total: 2.11s	remaining: 51.9ms
976:	learn: 0.2471151	total: 2.11s	remaining: 49.7ms
977:	learn: 0.2470555	total: 2.12s	remaining: 47.6ms
978:	learn: 0.2469536	total: 2.12s	remaining: 45.4ms
979:	learn: 0.2468975	total: 2.12s	remaining: 43.2ms
980:	learn: 0.2466118	total: 2.12s	remaining: 41.1ms
981:	learn: 0.2465362	total: 2.12s	remaining: 38.9ms
982:	learn: 0.2464968	total: 2.13s	remaining: 36.8ms
983:	learn: 0.2464084	total: 2.13s	remaining: 34.6ms
984:	learn: 0.2462902	total: 2.13s	remaining: 32.4ms
985:	learn: 0.2462566	total: 2.13s	remaining: 30.3ms
986:	learn: 0.2462064	total: 2.13s	remaining: 28.1ms
987:	learn: 0.2461655	total: 2.13s	remaining: 25.9ms
988:	learn: 0.2461125	total: 2.13s	remaining: 23.8ms
989:	learn: 0.2460279	total: 2.14s	remaining: 21

In [23]:
keys = list(res.keys())
values = list(res.values())
df_scores2 = pd.DataFrame({"Models":keys,"CV_Scores":values})

In [24]:
df_scores2

,Models,CV_Scores
0,Tree,0.820758
1,Forest,0.842071
2,Gb,0.824116
3,Xgb,0.821944
4,Log,0.797045
5,knn,0.811616
6,naive,0.830859
7,catboost,0.832096
8,lgbm,0.830783
